Note: original code has openai==1.23.6 but it has error so i used openai==0.28

In [1]:
!pip install python-dotenv==1.0.1 qdrant-client==1.9.0 openai==0.28 transformers==4.40.1 sentence-transformers==2.7.0 datasets==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 

In [2]:
from google.colab import userdata
QDRANT_API_KEY=userdata.get('QDRANT_API_KEY')
QDRANT_URL=userdata.get('QDRANT_URL')

In [ ]:
#print(QDRANT_API_KEY)
#print(QDRANT_URL)

In [7]:
from google.colab import userdata
OPENAI_API_KEY= userdata.get('OPENAI_API_KEY')

In [19]:
#print(OPENAI_API_KEY)

In [11]:
import openai

# Step 3: Set OpenAI API key securely
# You need to set the API key explicitly, replace with your actual key.
openai.api_key = OPENAI_API_KEY  # Replace with your actual API key

# Step 4: Define the embedding function
def get_text_embedding(
    text: str,
    model: str = "text-embedding-3-large"
) -> list:
    """
    Get the vector representation of the input text using the specified OpenAI embedding model.

    Args:
        text (str): The input text to be embedded.
        model (str, optional): The name of the OpenAI embedding model to use. Defaults to "text-embedding-3-large".

    Returns:
        list: The vector representation of the input text as a list of floats.

    Raises:
        openai.error.OpenAIError: If an error occurs during the API call.
    """
    try:
        response = openai.Embedding.create(
            input=text,
            model=model
        )
        embedding = response['data'][0]['embedding']
        return embedding
    except openai.error.OpenAIError as e:
        print(f"Error generating embedding: {e}")
        return []

# Step 5: Test the function
input_text = "This is a sample text to generate embeddings."
embedding_vector = get_text_embedding(input_text)
print("Embedding Vector:", embedding_vector)
print("Vector Length:", len(embedding_vector))


Embedding Vector: [-0.00021045458561275154, -0.014197973534464836, -0.013734831474721432, -0.012330218218266964, 0.01075097918510437, 0.002871862845495343, -0.0014653523685410619, 0.0338929146528244, 0.0077633303590118885, -0.003272367175668478, -0.0072090779431164265, 0.015526660718023777, 0.003014222253113985, -0.006370106711983681, 0.010971161536872387, -0.0032742652110755444, -0.00928562693297863, 0.004623832181096077, -0.007516574114561081, -0.01661238819360733, 0.0031698683742433786, 0.0040354132652282715, -0.05053567513823509, 0.0179182980209589, 0.03556326776742935, -0.004532721824944019, -0.014904076233506203, 0.01753867231309414, -0.0014112557983025908, 0.04746830463409424, -0.017326083034276962, -0.009361551143229008, 0.014289083890616894, -0.045008335262537, 0.030779991298913956, 0.009110999293625355, 0.028441501781344414, 0.04953346401453018, 0.011365971527993679, -0.009961359202861786, 0.050353456288576126, 0.00643843924626708, -0.036929916590452194, -0.006806675344705582

In [12]:
q_client.search(
    collection_name="arxiv_chunks",
    query_vector=("summary" ,get_text_embedding("machine learning in sound and diffusion")),
    with_payload=["summary", "title", "authors"],
    limit=2
)

[ScoredPoint(id='4b6c8500-63ec-4802-9e64-c11f8f0a50f8', version=0, score=0.413606, payload={'title': 'Diffsound: Discrete Diffusion Model for Text-to-sound Generation', 'authors': ['Dongchao Yang', 'Jianwei Yu', 'Helin Wang', 'Wen Wang', 'Chao Weng', 'Yuexian Zou', 'Dong Yu'], 'summary': 'Generating sound effects that humans want is an important topic. However,\nthere are few studies in this area for sound generation. In this study, we\ninvestigate generating sound conditioned on a text prompt and propose a novel\ntext-to-sound generation framework that consists of a text encoder, a Vector\nQuantized Variational Autoencoder (VQ-VAE), a decoder, and a vocoder. The\nframework first uses the decoder to transfer the text features extracted from\nthe text encoder to a mel-spectrogram with the help of VQ-VAE, and then the\nvocoder is used to transform the generated mel-spectrogram into a waveform. We\nfound that the decoder significantly influences the generation performance.\nThus, we focus

In [13]:
from qdrant_client import QdrantClient, models

exlusioner = models.PayloadSelectorExclude(exclude=["chunk", "text_id"])

q_client.search(
    collection_name="arxiv_chunks",
    query_vector=("summary" ,get_text_embedding("machine learning in sound and diffusion")),
    with_payload=exlusioner,
    limit=2
)

[ScoredPoint(id='4b6c8500-63ec-4802-9e64-c11f8f0a50f8', version=0, score=0.41360056, payload={'title': 'Diffsound: Discrete Diffusion Model for Text-to-sound Generation', 'source': 'http://arxiv.org/pdf/2207.09983', 'authors': ['Dongchao Yang', 'Jianwei Yu', 'Helin Wang', 'Wen Wang', 'Chao Weng', 'Yuexian Zou', 'Dong Yu'], 'summary': 'Generating sound effects that humans want is an important topic. However,\nthere are few studies in this area for sound generation. In this study, we\ninvestigate generating sound conditioned on a text prompt and propose a novel\ntext-to-sound generation framework that consists of a text encoder, a Vector\nQuantized Variational Autoencoder (VQ-VAE), a decoder, and a vocoder. The\nframework first uses the decoder to transfer the text features extracted from\nthe text encoder to a mel-spectrogram with the help of VQ-VAE, and then the\nvocoder is used to transform the generated mel-spectrogram into a waveform. We\nfound that the decoder significantly influen

In [14]:
author_filter = models.Filter(
    should=[
        models.FieldCondition(
            key="authors",
            match=models.MatchValue(value="Dong Yu")
            )
            ])

q_client.search(
    collection_name="arxiv_chunks",
    query_vector=("summary", get_text_embedding("machine learning in sound and diffusion")),
    query_filter=author_filter,
    limit=5
)

[ScoredPoint(id='4b6c8500-63ec-4802-9e64-c11f8f0a50f8', version=0, score=0.41373032, payload={'text_id': '4b6c8500-63ec-4802-9e64-c11f8f0a50f8', 'title': 'Diffsound: Discrete Diffusion Model for Text-to-sound Generation', 'source': 'http://arxiv.org/pdf/2207.09983', 'authors': ['Dongchao Yang', 'Jianwei Yu', 'Helin Wang', 'Wen Wang', 'Chao Weng', 'Yuexian Zou', 'Dong Yu'], 'chunk': 'that it can effectively alleviate the unidirectional bias and the\naccumulated prediction error problems. We adopt the idea\nfrom diffusion models, which use a forward process to corrupt\nthe original mel-spectrogram tokens in Tsteps, and then let the\nmodel learn to recover the original tokens in a reverse process.\nSpeciﬁcally, in the forward process, we deﬁne a transition\nmatrix that denotes probability of each token transfer to a\nrandom token or a pre-deﬁned MASK token. By using the\ntransition matrix, the original tokens x0\x18q(x0)transfer\ninto a stationary distribution p(xT). In the reverse proces

In [15]:
def search(
    named_vector_to_search: str,
    input_query: str,
    limit: int = 5,
    client: QdrantClient = q_client,
    collection_name: str = "arxiv_chunks",
    **kwargs):
    """
    Perform a vector search in the Qdrant database based on the input query.

    This method takes an input query string, converts it into a vector embedding using the
    "text-embedding-3-large" model, and searches for the closest matching vectors in the
    Qdrant database. The search results are returned as a list of dictionaries containing
    the item ID, similarity score, and payload information

    Args:
        input_query (str): The input query string to search for.
        named_vector_to_search: the vector you want to search against
        limit (int, optional): The maximum number of search results to return. Default is 3.
        kwargs: Additional keyword arguments to pass to the Qdrant search method.

    Returns:
        list: A list of dictionaries representing the search results. Each dictionary contains
              the following keys:
              - "id": The ID of the matching item in the Qdrant database.
              - "similarity_score": The similarity score between the input query and the matching item.
              - metadata from the payload

    """

    input_vector = get_text_embedding(input_query)

    search_result = client.search(
        collection_name=collection_name,
        query_vector=(named_vector_to_search, input_vector),
        limit=limit,
        with_payload=True,
        **kwargs
    )

    result = []
    for item in search_result:
        similarity_score = item.score
        payload = item.payload
        data = {
            "similarity_score": similarity_score,
            "summary": payload.get("summary"),
            "title": payload.get("title"),
            "source": payload.get("source"),
            "authors": payload.get("authors")
            }
        result.append(data)

    return result

In [16]:
QUERY_STRING = "agents, reasoning, chain-of-thought, few-shot prompting"

search(
    named_vector_to_search= "summary",
    input_query=QUERY_STRING
    )

[{'similarity_score': 0.58828825,
  'summary': 'The past decade has witnessed dramatic gains in natural language processing\nand an unprecedented scaling of large language models. These developments have\nbeen accelerated by the advent of few-shot techniques such as chain of thought\n(CoT) prompting. Specifically, CoT pushes the performance of large language\nmodels in a few-shot setup by augmenting the prompts with intermediate steps.\nDespite impressive results across various tasks, the reasons behind their\nsuccess have not been explored. This work uses counterfactual prompting to\ndevelop a deeper understanding of CoT-based few-shot prompting mechanisms in\nlarge language models. We first systematically identify and define the key\ncomponents of a prompt: symbols, patterns, and text. Then, we devise and\nconduct an exhaustive set of experiments across four different tasks, by\nquerying the model with counterfactual prompts where only one of these\ncomponents is altered. Our experim

In [17]:
search(
    named_vector_to_search= "summary",
    input_query=QUERY_STRING,
    score_threshold=0.51
    )

[{'similarity_score': 0.58838326,
  'summary': 'The past decade has witnessed dramatic gains in natural language processing\nand an unprecedented scaling of large language models. These developments have\nbeen accelerated by the advent of few-shot techniques such as chain of thought\n(CoT) prompting. Specifically, CoT pushes the performance of large language\nmodels in a few-shot setup by augmenting the prompts with intermediate steps.\nDespite impressive results across various tasks, the reasons behind their\nsuccess have not been explored. This work uses counterfactual prompting to\ndevelop a deeper understanding of CoT-based few-shot prompting mechanisms in\nlarge language models. We first systematically identify and define the key\ncomponents of a prompt: symbols, patterns, and text. Then, we devise and\nconduct an exhaustive set of experiments across four different tasks, by\nquerying the model with counterfactual prompts where only one of these\ncomponents is altered. Our experim

In [18]:
q_client.close()